In [9]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import messagebox, simpledialog
from PIL import Image, ImageTk

# User credentials
classmates = {
    "abdullah": {"matno": "21120613126"},
    "ada": {"matno": "21120612451"},
    "chima": {"matno": "21120612567"},
    "clement": {"matno": "21120612479"},
    "daniel": {"matno": "21120612478"},
    "emeka": {"matno": "21120612449"},
    "ilyas": {"matno": "21120612448"},
    "miracle": {"matno": "21120612475"},
    "tabitha": {"matno": "21120612496"},
    "tireni": {"matno": "21120613127"},
}

# Image directories
image_dirs = {
    "Traditional": "img/traditional art",
    "Modern": "img/modern art",
    "Contemporary": "img/contemporary art"
}

# GUI Setup
window = tk.Tk()
window.title("CA for Image Enhancement")
window.geometry('600x600')
window.configure(bg='#333333')

current_user = None
selected_category = None
selected_artwork = None

def login():
    username = username_entry.get().strip().lower()
    password = password_entry.get()
    if username in classmates and password == classmates[username]["matno"]:
        global current_user
        current_user = username
        messagebox.showinfo("Login Success", f"Welcome, {username.title()}!")
        show_category_frame()
    else:
        messagebox.showerror("Login Failed", "Invalid username or matric number.")

def show_login_frame():
    for widget in window.winfo_children():
        widget.destroy()
    frame = tk.Frame(window, bg='#333333')
    
    tk.Label(frame, text="Login", bg='#333333', fg="#4338CA", font=("Arial", 24)).grid(row=0, column=0, columnspan=2, pady=20)
    
    tk.Label(frame, text="Username:", bg='#333333', fg="#FFFFFF", font=("Arial", 14)).grid(row=1, column=0, padx=10, pady=10, sticky='e')
    global username_entry
    username_entry = tk.Entry(frame, font=("Arial", 14))
    username_entry.grid(row=1, column=1, padx=10, pady=10)
    
    tk.Label(frame, text="Matric Number:", bg='#333333', fg="#FFFFFF", font=("Arial", 14)).grid(row=2, column=0, padx=10, pady=10, sticky='e')
    global password_entry
    password_entry = tk.Entry(frame, font=("Arial", 14), show="*")
    password_entry.grid(row=2, column=1, padx=10, pady=10)
    
    tk.Button(frame, text="Login", bg="#4338CA", fg="#FFFFFF", font=("Arial", 16), command=login).grid(row=3, column=0, columnspan=2, pady=30)
    
    frame.pack()

def show_category_frame():
    for widget in window.winfo_children():
        widget.destroy()
    frame = tk.Frame(window, bg='#333333')
    tk.Label(frame, text="Select Art Category", bg='#333333', fg="#4338CA", font=("Arial", 18)).pack(pady=20)
    for category in image_dirs.keys():
        tk.Button(frame, text=category, bg="#4338CA", fg="#FFFFFF", font=("Arial", 14), command=lambda c=category: show_artwork_frame(c)).pack(pady=5)
    tk.Button(frame, text="Back", bg="red", fg="white", font=("Arial", 14), command=show_login_frame).pack(pady=20)
    frame.pack()

def show_artwork_frame(category):
    global selected_category
    selected_category = category
    for widget in window.winfo_children():
        widget.destroy()
    frame = tk.Frame(window, bg='#333333')
    tk.Label(frame, text=f"Select {category} Artwork", bg='#333333', fg="#4338CA", font=("Arial", 18)).pack(pady=20)
    
    image_path = image_dirs.get(category, "")
    if os.path.exists(image_path):
        images = [f for f in os.listdir(image_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        for img in images:
            tk.Button(frame, text=img, bg="#4338CA", fg="#FFFFFF", font=("Arial", 14), command=lambda i=img: show_enhancement_frame(image_path, i)).pack(pady=5)
    
    tk.Button(frame, text="Back", bg="red", fg="#FFFFFF", font=("Arial", 14), command=show_category_frame).pack(pady=20)
    frame.pack()

def show_enhancement_frame(image_path, image_name):
    global selected_artwork
    selected_artwork = os.path.join(image_path, image_name)
    for widget in window.winfo_children():
        widget.destroy()
    frame = tk.Frame(window, bg='#333333')
    tk.Label(frame, text="Select Enhancement", bg='#333333', fg="#4338CA", font=("Arial", 18)).pack(pady=20)
    
    detection_operations = {
        "Sobel Edge Detection": sobel_detection,
        "Prewitt Edge Detection": prewitt_detection,
        "Robert Edge Detection": robert_detection,
        "Laplacian Edge Detection": laplacian_detection,
        "Canny Edge Detection": canny_detection
    }
    
    image = cv2.imread(selected_artwork)
    for enh, func in detection_operations.items():
        tk.Button(frame, text=enh, bg="#4338CA", fg="#FFFFFF", font=("Arial", 14), command=lambda f=func: process_image(image, f)).pack(pady=5)
    
    tk.Button(frame, text="Back", bg="red", fg="#FFFFFF", font=("Arial", 14), command=lambda: show_artwork_frame(selected_category)).pack(pady=20)
    frame.pack()

def display_images(original, enhanced):
    for widget in window.winfo_children():
        widget.destroy()
    
    frame = tk.Frame(window, bg='#333333')
    frame.pack()
    
    original_img = Image.fromarray(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    enhanced_img = Image.fromarray(cv2.cvtColor(enhanced, cv2.COLOR_BGR2RGB))
    
    original_img = ImageTk.PhotoImage(original_img.resize((250, 250)))
    enhanced_img = ImageTk.PhotoImage(enhanced_img.resize((250, 250)))
    
    tk.Label(frame, text="Original Image", bg='#333333', fg='#FFFFFF', font=("Arial", 14)).grid(row=0, column=0, padx=10, pady=10)
    tk.Label(frame, text="Edge Detection Image", bg='#333333', fg='#FFFFFF', font=("Arial", 14)).grid(row=0, column=1, padx=10, pady=10)
    
    tk.Label(frame, image=original_img).grid(row=1, column=0, padx=10, pady=10)
    tk.Label(frame, image=enhanced_img).grid(row=1, column=1, padx=10, pady=10)
    
    # Keep references to avoid garbage collection
    frame.image1 = original_img
    frame.image2 = enhanced_img

    # tk.Button(
    #     frame, text="Back", bg="red", fg="white", font=("Arial", 14),
    #     command=lambda: show_enhancement_frame(os.path.dirname(selected_artwork), os.path.basename(selected_artwork))
    # ).grid(row=2, column=0, columnspan=2, pady=20)

    # tk.Button(
    #     frame, text="Log Out", bg="#4338CA", fg="white", font=("Arial", 14),
    #     command=lambda: show_login_frame()
    # ).grid(row=2, column=0, columnspan=2, pady=20
        # Back Button
    tk.Button(
        frame, text="Back", bg="red", fg="white", font=("Arial", 14),
        command=lambda: show_enhancement_frame(os.path.dirname(selected_artwork), os.path.basename(selected_artwork))
    ).grid(row=2, column=0, pady=20)

    # Logout Button (if needed)
    tk.Button(
        frame, text="Logout", bg="#6B7280", fg="white", font=("Arial", 14),
        command=show_login_frame
    ).grid(row=2, column=1, pady=20)



def process_image(image, function):
    enhanced_image = function(image)
    # Convert float images (e.g., from Sobel or Laplacian) to uint8 for display
    if enhanced_image.dtype != np.uint8:
        enhanced_image = cv2.convertScaleAbs(enhanced_image)
    display_images(image, enhanced_image)

# Edge Detection Techniques    
def sobel_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    sobel_combined = cv2.magnitude(sobel_x, sobel_y)
    
    return np.uint8(sobel_combined)

def prewitt_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kernel_x = np.array([[-1, -1, -1],
                         [0,  0,  0],
                         [1,  1,  1]])
    
    kernel_y = np.array([[-1, 0, 1],
                         [-1, 0, 1],
                         [-1, 0, 1]])

    prewitt_x = cv2.filter2D(gray, -1, kernel_x)
    prewitt_y = cv2.filter2D(gray, -1, kernel_y)

    prewitt_combined = np.sqrt(np.square(prewitt_x) + np.square(prewitt_y))

    return np.uint8(prewitt_combined)

def robert_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kernel_x = np.array([[1, 0],
                         [0, -1]])
    kernel_y = np.array([[0, 1],
                         [-1, 0]])

    robert_x = cv2.filter2D(gray, -1, kernel_x)
    robert_y = cv2.filter2D(gray, -1, kernel_y)

    robert_combined = np.sqrt(np.square(robert_x) + np.square(robert_y))

    return np.uint8(robert_combined)

def laplacian_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    laplacian = cv2.convertScaleAbs(laplacian)  # Convert to 8-bit

    return laplacian

def canny_detection(image):
    return cv2.Canny(image, 100, 200)

# Launch the login screen
show_login_frame()
window.mainloop()